In [29]:
from dotenv import load_dotenv

load_dotenv()

True

## initialize_agent


In [47]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType


# Initialize our OpenAI model—we'll use temperature=0 for predictable outputs
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [48]:
from langchain.tools import StructuredTool


# Let's create a simple calculator tool as an example:
def calculator(query: str) -> float:
    """Evaluates simple math expressions like '5 + 3'."""
    return eval(query)


calc_tool = StructuredTool.from_function(
    func=calculator,
    name="calculator",
    description="Useful for evaluating math expressions such as addition, substraction, multiplication and division.",
)

# Create agent quickly with initialize_agent
agent = initialize_agent(
    tools=[calc_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Let's ask a simple question to our agent
result = agent.invoke("What's 15 multiplied by 7, then added to 10?")
print("\nResult with initialize_agent:", result["output"])



> Entering new AgentExecutor chain...
To solve the problem, I need to first multiply 15 by 7, and then add 10 to the result. 

Action: calculator
Action Input: "15 * 7"
Observation: 105
Thought:Now that I have the result of the multiplication, I need to add 10 to it.

Action: calculator  
Action Input: "105 + 10"  
Observation: 115
Thought:I now know the final answer.  
Final Answer: 115

> Finished chain.

Result with initialize_agent: 115


## create_react_agent


In [49]:
from langchain.agents import AgentExecutor
from langchain.agents.react.agent import create_react_agent
from langchain_core.prompts import PromptTemplate

In [66]:
# Custom prompt (feel free to tweak as you like)
react_prompt = PromptTemplate.from_template(
    """
        Answer the following questions clearly and step-by-step.

        You have access to these tools:
        {tools}

        Use this format strictly:

        Question: {input}
        Thought: Consider what's next logically
        Action: Select a suitable tool from {tool_names}
        Action Input: Provide relevant input like this "num1": <given number 1>, "num2": <given number 2>
        Observation: Examine the tool's response, and if something wrong mention that as well
        
        Thought: I've got what I need now
        Final Answer: Clearly state the final answer here.

        {agent_scratchpad}
    """
)

In [67]:
from pydantic import BaseModel, Field


class CalculatorModel(BaseModel):
    num1: float = Field(description="first integer number")
    num2: float = Field(description="second integer number")


# Let's create a simple calculator tool as an example:
def multiplication_with_schema(num1: float, num2: float) -> float:
    """Evaluates simple math expressions like '5 * 3'."""
    return num1 * num2


multiplication_tool_with_schema = StructuredTool.from_function(
    func=multiplication_with_schema,
    name="multiplication_with_schema",
    description="Useful for evaluating multiplication",
    args_schema=CalculatorModel,
    return_direct=True,
)


# Let's create a simple calculator tool as an example:
def addition_with_schema(num1: float, num2: float) -> float:
    """Evaluates simple math expressions like '5 + 3'."""
    return num1 + num2


addition_tool_with_schema = StructuredTool.from_function(
    func=addition_with_schema,
    name="addition_with_schema",
    description="Useful for evaluating addition",
    args_schema=CalculatorModel,
    return_direct=True,
)

In [68]:
# Explicitly create our customized react agent with specific tools and prompt
react_agent = create_react_agent(
    llm=llm,
    tools=[multiplication_tool_with_schema, addition_tool_with_schema],
    prompt=react_prompt,
)

# Manually create AgentExecutor for execution
executor = AgentExecutor(
    agent=react_agent,
    tools=[multiplication_tool_with_schema, addition_tool_with_schema],
    verbose=True,
    max_iterations=5,
)

# Now let's ask our agent the same math question
result_custom = executor.invoke({"input": "What's 15 multiplied by 7, then added to 10?"})
print("\nResult with create_react_agent:", result_custom["output"])



> Entering new AgentExecutor chain...
Question: What's 15 multiplied by 7, then added to 10?  
Thought: First, I need to multiply 15 by 7, and then I will add 10 to the result.  
Action: Select a suitable tool from multiplication_with_schema  
Action Input: "num1": 15, "num2": 7  Select a suitable tool from multiplication_with_schema is not a valid tool, try one of [multiplication_with_schema, addition_with_schema].Question: What's 15 multiplied by 7, then added to 10?  
Thought: First, I need to multiply 15 by 7, and then I will add 10 to the result.  
Action: Select a suitable tool from multiplication_with_schema  
Action Input: "num1": 15, "num2": 7  Select a suitable tool from multiplication_with_schema is not a valid tool, try one of [multiplication_with_schema, addition_with_schema].Question: What's 15 multiplied by 7, then added to 10?  
Thought: First, I need to multiply 15 by 7, and then I will add 10 to the result.  
Action: Select a suitable tool from multiplication_with_s